In [ ]:

class Module(name: String, type: Char, outputs: List<String>) {
    val name: String = name
    val type: Char = type
    val outputs: List<String> = outputs
    var memory: Any = if (type == '%') "off" else mutableMapOf<String, String>()

    override fun toString(): String {
        return "$name{type=$type,outputs=${outputs.joinToString()},memory=$memory}"
    }
}

val modules = HashMap<String, Module>()
var broadcastTargets = listOf<String>()

File("input.txt").forEachLine { line ->
    val (left, right) = line.trim().split(" -> ")
    val outputs = right.split(", ")
    if (left == "broadcaster") {
        broadcastTargets = outputs
    } else {
        val type = left[0]
        val name = left.substring(1)
        modules[name] = Module(name, type, outputs)
    }
}

modules.forEach { (name, module) ->
    module.outputs.forEach { output ->
        if (output in modules && modules[output]?.type == '&') {
            @Suppress("UNCHECKED_CAST")
            val moduleMemory = modules[output]!!.memory as HashMap<String, String>
            moduleMemory[name] = "lo"
        }
    }
}

var lo = 0
var hi = 0

repeat(1000) {
    lo++
    val q = ArrayDeque(broadcastTargets.map { Triple("broadcaster", it, "lo") })

    while (q.isNotEmpty()) {
        val (origin, target, pulse) = q.removeFirst()

        if (pulse == "lo") {
            lo++
        } else {
            hi++
        }

        if (target !in modules) continue

        val module = modules[target]!!

        if (module.type == '%') {
            if (pulse == "lo") {
                module.memory = if (module.memory == "off") "on" else "off"
                val outgoing = if (module.memory == "on") "hi" else "lo"
                module.outputs.forEach { x ->
                    q.add(Triple(module.name, x, outgoing))
                }
            }
        } else {
            @Suppress("UNCHECKED_CAST")
            val moduleMemory = module.memory as MutableMap<String, String>
            moduleMemory[origin] = pulse
            val outgoing = if (moduleMemory.values.all { it == "hi" }) "lo" else "hi"
            module.outputs.forEach { x ->
                q.add(Triple(module.name, x, outgoing))
            }
        }
    }
}

lo * hi

In [ ]:
import java.io.File
import kotlin.system.exitProcess

class Module(name: String, type: Char, outputs: List<String>) {
    val name: String = name
    val type: Char = type
    val outputs: List<String> = outputs
    var memory: Any = if (type == '%') "off" else mutableMapOf<String, String>()

    override fun toString(): String {
        return "$name{type=$type,outputs=${outputs.joinToString()},memory=$memory}"
    }
}

val modules = mutableMapOf<String, Module>()
var broadcastTargets = listOf<String>()

File("input.txt").forEachLine { line ->
    val (left, right) = line.split(" -> ")
    val outputs = right.split(", ")
    if (left == "broadcaster") {
        broadcastTargets = outputs
    } else {
        val type = left[0]
        val name = left.substring(1)
        modules[name] = Module(name, type, outputs)
    }
}

modules.forEach { (name, module) ->
    module.outputs.forEach { output ->
        if (output in modules && modules[output]?.type == '&') {
            @Suppress("UNCHECKED_CAST")
            val moduleMemory = modules[output]!!.memory as HashMap<String, String>
            moduleMemory[name] = "lo"
        }
    }
}

val feed = modules.filterValues { "rx" in it.outputs }.keys.first()
val cycleLengths = mutableMapOf<String, Long>()
val seen = modules.filterValues { feed in it.outputs }.keys.associateWith { 0 }.toMutableMap()

var presses = 0L

outerloop@while (true) {
    presses++
    val q: ArrayDeque<Triple<String, String, String>> = ArrayDeque()
    broadcastTargets.forEach { target ->
        q.add(Triple("broadcaster", target, "lo"))
    }

    while (q.isNotEmpty()) {
        val (origin, target, pulse) = q.removeFirst()

        if (target !in modules) continue

        val module = modules[target]!!

        if (module.name == feed && pulse == "hi") {
            seen[origin] = seen.getOrDefault(origin, 0) + 1

            if (cycleLengths[origin] == null) {
                cycleLengths[origin] = presses
            } else {
                assert(presses == seen[origin]!! * cycleLengths[origin]!!)
            }

            if (seen.values.all { it > 0 }) {
                var x = 1L
                cycleLengths.values.forEach { cycleLength ->
                    x = x * cycleLength / gcd(x, cycleLength)
                }
                println(x)
                break@outerloop
            }
        }

        if (module.type == '%') {
            if (pulse == "lo") {
                module.memory = if (module.memory == "off") "on" else "off"
                val outgoing = if (module.memory == "on") "hi" else "lo"
                module.outputs.forEach { x ->
                    q.add(Triple(module.name, x, outgoing))
                }
            }
        } else {
            @Suppress("UNCHECKED_CAST")
            val moduleMemory = module.memory as HashMap<String, String>
            moduleMemory[origin] = pulse
            val outgoing = if (moduleMemory.values.all { it == "hi" }) "lo" else "hi"
            module.outputs.forEach { x ->
                q.add(Triple(module.name, x, outgoing))
            }
        }
    }
}


fun gcd(a: Long, b: Long): Long {
    return if (b == 0L) a else gcd(b, a % b)
}
